In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

## 데이터 수정

In [3]:
for df in [df_train,df_test]:
    # df['date'] = df['date'].apply(lambda x: x[0:8])
    # 일짜까지 되있는걸 월까지
    df['date'] = df['date'].apply(lambda x: x[0:6])
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: np.nan if x == 0 else x)
    df['yr_renovated'] = df['yr_renovated'].fillna(df['yr_built'])
for df in [df_train]:
    # 1000단위 대로 반올림
    df['price'] = df['price'].apply(lambda x: round(x, -3))

In [4]:
for df in [df_train,df_test]:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')

In [5]:
df_train['per_price'] = df_train['price']/df_train['sqft_total_size']
zipcode_price = df_train.groupby(['zipcode'])['per_price'].agg({'mean','var'}).reset_index()
df_train = pd.merge(df_train,zipcode_price,how='left',on='zipcode')
df_test = pd.merge(df_test,zipcode_price,how='left',on='zipcode')

for df in [df_train,df_test]:
    df['mean'] = df['mean'] * df['sqft_total_size']
    df['var'] = df['var'] * df['sqft_total_size']

### 상호작용변수 추가

In [6]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

### 월별 + ZIPCODE를 활용한 더미변수 추가하여 테스트
위도 + 경도로 위치값을 대신하고 싶었지만 어떻게 구역을 나눠야 할지 모르기때문에 ZIPCODE로 대신함

In [7]:
df_train_length = len(df_train)
concat_dataset = pd.concat(objs=[df_train, df_test], axis=0)

# LabelEncoder
LE.fit(df_train[['date']])
concat_dataset[['date']] = LE.transform(concat_dataset[['date']])
LE.fit(df_train[['zipcode']])
concat_dataset[['zipcode']] = LE.transform(concat_dataset[['zipcode']])
concat_dataset['date_zipcode'] = concat_dataset['date'].map(str) + "_" + concat_dataset['zipcode'].map(str)
LE.fit(concat_dataset[['date_zipcode']])
concat_dataset[['date_zipcode']] = LE.transform(concat_dataset[['date_zipcode']])

dataset_preprocessed = pd.get_dummies(concat_dataset, columns=['date', 'zipcode','date_zipcode'])

df_train = dataset_preprocessed[:df_train_length]
df_test = dataset_preprocessed[df_train_length:]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## XGboost

In [8]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import xgboost as xgb
from xgboost import XGBClassifier

In [9]:
# train_columns = [c for c in df_train.columns if c not in ['id','price','per_price','lat','long']]
train_columns = [c for c in df_train.columns if c not in ['id','price','per_price']]
y_reg = df_train['price']

In [10]:
model= xgb.XGBRegressor(
    learning_rate=0.005,
    # 모형 갯수 default=100
    n_estimators=5000,
    random_state=2019,
    num_boost_round=20000, 
    early_stopping_rounds=4000,
    verbose_eval=3000, 
    show_stdv=False,
    max_depth=30, 
    nthread=-1
)

#prepare fit model with cross-validation
folds = KFold(n_splits=4, shuffle=True, random_state=2019)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train)):
    X_train,X_validation=df_train.iloc[trn_idx][train_columns],df_train.iloc[val_idx][train_columns]
    y_train,y_validation=y_reg.iloc[trn_idx],y_reg.iloc[val_idx]
    
    dtrain = xgb.DMatrix(X_train)
    dtarget=xgb.DMatrix(y_train)
    dtest = xgb.DMatrix(X_validation)
    dtest_target=xgb.DMatrix(y_validation)
    
    model.fit(X_train,y_train
              ,eval_set=[(X_train,y_train),(X_validation,y_validation)]
              ,eval_metric='rmse'
              ,verbose=1000)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	validation_0-rmse:645999	validation_1-rmse:673407
[1000]	validation_0-rmse:12617.8	validation_1-rmse:140263
[2000]	validation_0-rmse:929.247	validation_1-rmse:138880
[3000]	validation_0-rmse:275.519	validation_1-rmse:138838
[3999]	validation_0-rmse:149.454	validation_1-rmse:138834
[0]	validation_0-rmse:651669	validation_1-rmse:656784
[1000]	validation_0-rmse:13041.6	validation_1-rmse:131180
[2000]	validation_0-rmse:1043.77	validation_1-rmse:130362
[3000]	validation_0-rmse:288.607	validation_1-rmse:130348
[3999]	validation_0-rmse:178.073	validation_1-rmse:130346
[0]	validation_0-rmse:656977	validation_1-rmse:640700
[1000]	validation_0-rmse:13713.9	validation_1-rmse:120550
[2000]	validation_0-rmse:1200.22	validation_1-rmse:119814


KeyboardInterrupt: 

In [ ]:
predictions=model.predict(df_test[train_columns])

submission = pd.read_csv("sample_submission.csv")
submission['price']=predictions
submission.to_csv('submission.csv',index=False)